# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
from skimage import transform
import datetime
import os
import warnings
warnings.filterwarnings("ignore")

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())


In [4]:
img_width = 100
img_height = 100
zoom = 0.8
batch_size = 20
num_classes = 5
num_epochs = 20
img_indices = [x for x in range(30)]
inputShape = (len(img_indices), img_width, img_height, 3)

### Transform image

In [5]:
def cropimread(img_pre ,xcrop, ycrop):
    ysize, xsize, chan = img_pre.shape
    xoff = (xsize - xcrop) // 2
    yoff = (ysize - ycrop) // 2
    img= img_pre[yoff:-yoff,xoff:-xoff]
    return img

def transform_img(img, x, y):
    # Crop imgs
    #img = cropimread(img, x, y)
    
    # Resize
    img = transform.resize(img, (x, y))
    
    # Normalize
    img = img / 255
    
    return img

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [6]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    # Choose even indexed images
    img_idx = img_indices
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t) // batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),img_width,img_height,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,num_classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate over the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    # specify x and y coordinates to be used for shifting (mid points)
                    image = transform_img(image, img_width, img_height)

                    batch_data[folder,idx,:,:,0] = image[:, :, 0]
                    batch_data[folder,idx,:,:,1] = image[:, :, 1]
                    batch_data[folder,idx,:,:,2] = image[:, :, 2]
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        last_batch_size = len(t) % batch_size
        if last_batch_size > 1:
            last_batch_data = np.zeros((last_batch_size,len(img_idx),img_width,img_height,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            last_batch_labels = np.zeros((last_batch_size,num_classes)) # batch_labels is the one hot representation of the output
            for folder in range(last_batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches * batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate over the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    # specify x and y coordinates to be used for shifting (mid points)
                    image = transform_img(image, img_width, img_height)

                    last_batch_data[folder,idx,:,:,0] = image[:, :, 0]
                    last_batch_data[folder,idx,:,:,1] = image[:, :, 1]
                    last_batch_data[folder,idx,:,:,2] = image[:, :, 2]

                    last_batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
            yield last_batch_data, last_batch_labels
            

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [7]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

# training sequences = 663
# validation sequences = 100


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [8]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.regularizers import l2
#write your model here

### Conv3D 

In [9]:
K.set_image_data_format('channels_last')

model = Sequential()
model.add(Conv3D(32, (3, 3, 3), padding='same',
                 input_shape=inputShape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(3, 3, 3)))
model.add(Dropout(0.25))

model.add(Conv3D(64, (3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(128, (3, 3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,kernel_regularizer=l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [10]:
optimiser = "sgd"
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 100, 100, 32)  2624      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 100, 100, 32)  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 100, 100, 32)  128       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 33, 33, 32)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 33, 33, 32)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 10, 33, 33, 64)    55360     
_________________________________________________________________
activation_2 (Activation)    (None, 10, 33, 33, 64)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [11]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [12]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [14]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/20
34/34 [==============================] - 254s 7s/step - loss: 12.9030 - categorical_accuracy: 0.3679 - val_loss: 11.0848 - val_categorical_accuracy: 0.3900

Epoch 00001: saving model to model_init_2020-03-2317_15_15.245836/model-00001-12.94957-0.36048-11.08482-0.39000.h5
Epoch 2/20
34/34 [==============================] - 121s 4s/step - loss: 10.7440 - categorical_accuracy: 0.4893 - val_loss: 10.3603 - val_categorical_accuracy: 0.6000

Epoch 00002: saving model to model_init_2020-03-2317_15_15.245836/model-00002-10.74062-0.49321-10.36026-0.60000.h5
Epoch 3/20
34/34 [==============================] - 106s 3s/step - loss: 10.2864 - categorical_accuracy: 0.5909 - val_loss: 10.3629 - val_categorical_accuracy: 0.5300

Epoch 00003: saving model to model_init_2020-03-2317_15_15.245836/model-00003-10.27387-0.59729-10.36291-0.53000.h5
Epoch 4/20
34/34 [===========================